# Household Electric Power Consumption Analysis & Forecasting with ARIMA, SARIMA, Prophet, RNN, LSTM, GRU Models
---

## Problem Definition

Predict how much energy will have to be produced in December 2010 to serve customers.

We have data from the year of December 2006 to the year of November 2010. The column Global_active_power is the target.

## About Dataset

**Data Set Information:**

This archive contains 2075259 measurements gathered between December 2006 and November 2010 (47 months).
Notes:

- 1. (global_active_power*1000/60 - sub_metering_1 - sub_metering_2 - sub_metering_3) represents the active energy consumed every minute (in watt hour) in the household by electrical equipment not measured in sub-meterings 1, 2 and 3.

- 2. The dataset contains some missing values in the measurements (nearly 1,25% of the rows). All calendar timestamps are present in the dataset but for some timestamps, the measurement values are missing: a missing value is represented by the absence of value between two consecutive semi-colon attribute separators. For instance, the dataset shows missing values on April 28, 2007.

**Attribute Information:**
- 1. date: Date in format dd/mm/yyyy

- 2. time: time in format hh:mm:ss

- 3. global_active_power: household global minute-averaged active power (in kilowatt)

- 4. global_reactive_power: household global minute-averaged reactive power (in kilowatt)

- 5. voltage: minute-averaged voltage (in volt)

- 6. global_intensity: household global minute-averaged current intensity (in ampere)

- 7. sub_metering_1: energy sub-metering No. 1 (in watt-hour of active energy). It corresponds to the kitchen, containing mainly a dishwasher, an oven and a microwave (hot plates are not electric but gas powered).

- 8. sub_metering_2: energy sub-metering No. 2 (in watt-hour of active energy). It corresponds to the laundry room, containing a washing-machine, a tumble-drier, a refrigerator and a light.

- 9. sub_metering_3: energy sub-metering No. 3 (in watt-hour of active energy). It corresponds to an electric water-heater and an air-conditioner.

## Data Preparation

### Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

import plotly.express as px
import plotly.graph_objects as go
import cufflinks as cf

import warnings

# Suppress warnings
warnings.filterwarnings("ignore")

# Set default figure size for matplotlib
plt.rcParams["figure.figsize"] = (10, 6)

# Set Seaborn style
sns.set_style("whitegrid")

# Set pandas display options
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_columns', None)  # Display all columns in the dataframe

# Uncomment the line below if you want to display all rows in the dataframe
# pd.set_option('display.max_rows', None)

# Garbage collection for memory management
import gc

### Load Data

In [ ]:
# Install Kaggle package
!pip install -q kaggle

# Create the Kaggle directory if it doesn't exist
import os

kaggle_dir = os.path.expanduser('~/.kaggle')
if not os.path.exists(kaggle_dir):
    os.makedirs(kaggle_dir)

# Copy the kaggle.json file into the Kaggle directory
!cp kaggle.json ~/.kaggle/

# Set permissions for the kaggle.json file
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset from Kaggle
!kaggle datasets download -d uciml/electric-power-consumption-data-set

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/uciml/electric-power-consumption-data-set
License(s): DbCL-1.0
 26% 5.00M/19.4M [00:00<00:00, 29.7MB/s]
100% 19.4M/19.4M [00:00<00:00, 78.1MB/s]


In [ ]:
# Unzip the downloaded dataset
!unzip -q '/content/electric-power-consumption-data-set.zip' -d '/content/dataset/'

In [ ]:
# Path to the dataset
file_path = '/content/dataset/household_power_consumption.txt'

# Read the CSV file, parse 'date' column as datetime, and set it as index
df = pd.read_csv(file_path,
                sep=';',
                parse_dates={'Date_time' : ['Date', 'Time']},  # Convert 'date_hora' column to datetime
                # index_col='Date_time',     # Set 'date' column as index
                infer_datetime_format=True,
                low_memory=False,
                na_values=['nan', '?'])

### Explore Data

In [ ]:
# Display the dataframe
df

,Date_time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,2006-12-16 17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.000
1,2006-12-16 17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.000
2,2006-12-16 17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.000
3,2006-12-16 17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.000
4,2006-12-16 17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.000
...,...,...,...,...,...,...,...,...
2075254,2010-11-26 20:58:00,0.946,0.000,240.430,4.000,0.000,0.000,0.000
2075255,2010-11-26 20:59:00,0.944,0.000,240.000,4.000,0.000,0.000,0.000
2075256,2010-11-26 21:00:00,0.938,0.000,239.820,3.800,0.000,0.000,0.000
2075257,2010-11-26 21:01:00,0.934,0.000,239.700,3.800,0.000,0.000,0.000


In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075259 entries, 0 to 2075258
Data columns (total 8 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   Date_time              datetime64[ns]
 1   Global_active_power    float64       
 2   Global_reactive_power  float64       
 3   Voltage                float64       
 4   Global_intensity       float64       
 5   Sub_metering_1         float64       
 6   Sub_metering_2         float64       
 7   Sub_metering_3         float64       
dtypes: datetime64[ns](1), float64(7)
memory usage: 126.7 MB
None


In [ ]:
# Display descriptive statistics of the dataframe, transposed for better readability
print(df.describe().T)

                            count                           mean  \
Date_time                 2075259  2008-12-06 07:12:59.999994112   
Global_active_power   2049280.000                          1.092   
Global_reactive_power 2049280.000                          0.124   
Voltage               2049280.000                        240.840   
Global_intensity      2049280.000                          4.628   
Sub_metering_1        2049280.000                          1.122   
Sub_metering_2        2049280.000                          1.299   
Sub_metering_3        2049280.000                          6.458   

                                       min                  25%  \
Date_time              2006-12-16 17:24:00  2007-12-12 00:18:30   
Global_active_power                  0.076                0.308   
Global_reactive_power                0.000                0.048   
Voltage                            223.200              238.990   
Global_intensity                     0.200          

In [ ]:
print(df.head())

            Date_time  Global_active_power  Global_reactive_power  Voltage  \
0 2006-12-16 17:24:00                4.216                  0.418  234.840   
1 2006-12-16 17:25:00                5.360                  0.436  233.630   
2 2006-12-16 17:26:00                5.374                  0.498  233.290   
3 2006-12-16 17:27:00                5.388                  0.502  233.740   
4 2006-12-16 17:28:00                3.666                  0.528  235.680   

   Global_intensity  Sub_metering_1  Sub_metering_2  Sub_metering_3  
0            18.400           0.000           1.000          17.000  
1            23.000           0.000           1.000          16.000  
2            23.000           0.000           2.000          17.000  
3            23.000           0.000           1.000          17.000  
4            15.800           0.000           1.000          17.000  


### Create new features based on the datetime index

In [ ]:
data = df.copy()

In [ ]:
# Convert Time column to datetime
data['Date_time'] = pd.to_datetime(data['Date_time'], format='%Y%m%d:%H%M')

In [ ]:
# Create new features based on the datetime index
data['Year'] = data['Date_time'].dt.year
data['Month'] = data['Date_time'].dt.month
data['Day'] = data['Date_time'].dt.day
data['Hour'] = data['Date_time'].dt.hour
data['DayOfWeek'] = data['Date_time'].dt.dayofweek
data['Month'] = data['Date_time'].dt.month

In [ ]:
# Display the first few rows of the dataframe with new features
data.head()

,Date_time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,Year,Month,Day,Hour,DayOfWeek
0,2006-12-16 17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.000,2006,12,16,17,5
1,2006-12-16 17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.000,2006,12,16,17,5
2,2006-12-16 17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.000,2006,12,16,17,5
3,2006-12-16 17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.000,2006,12,16,17,5
4,2006-12-16 17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.000,2006,12,16,17,5


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075259 entries, 0 to 2075258
Data columns (total 13 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   Date_time              datetime64[ns]
 1   Global_active_power    float64       
 2   Global_reactive_power  float64       
 3   Voltage                float64       
 4   Global_intensity       float64       
 5   Sub_metering_1         float64       
 6   Sub_metering_2         float64       
 7   Sub_metering_3         float64       
 8   Year                   int32         
 9   Month                  int32         
 10  Day                    int32         
 11  Hour                   int32         
 12  DayOfWeek              int32         
dtypes: datetime64[ns](1), float64(7), int32(5)
memory usage: 166.2 MB


In [ ]:
# Finding out if the day is the weekend
def is_weekend(data):
    if data.dayofweek == 5 or data.dayofweek == 6:
        return 1 # weekened
    else:
        return 0 # working_day

data['Weekend'] = data['Date_time'].apply(is_weekend)
print(data.head())

            Date_time  Global_active_power  Global_reactive_power  Voltage  \
0 2006-12-16 17:24:00                4.216                  0.418  234.840   
1 2006-12-16 17:25:00                5.360                  0.436  233.630   
2 2006-12-16 17:26:00                5.374                  0.498  233.290   
3 2006-12-16 17:27:00                5.388                  0.502  233.740   
4 2006-12-16 17:28:00                3.666                  0.528  235.680   

   Global_intensity  Sub_metering_1  Sub_metering_2  Sub_metering_3  Year  \
0            18.400           0.000           1.000          17.000  2006   
1            23.000           0.000           1.000          16.000  2006   
2            23.000           0.000           2.000          17.000  2006   
3            23.000           0.000           1.000          17.000  2006   
4            15.800           0.000           1.000          17.000  2006   

   Month  Day  Hour  DayOfWeek  Weekend  
0     12   16    17       

In [ ]:
# Set Date_time as index number
data = data.set_index('Date_time')
print(data.head())

                     Global_active_power  Global_reactive_power  Voltage  \
Date_time                                                                  
2006-12-16 17:24:00                4.216                  0.418  234.840   
2006-12-16 17:25:00                5.360                  0.436  233.630   
2006-12-16 17:26:00                5.374                  0.498  233.290   
2006-12-16 17:27:00                5.388                  0.502  233.740   
2006-12-16 17:28:00                3.666                  0.528  235.680   

                     Global_intensity  Sub_metering_1  Sub_metering_2  \
Date_time                                                               
2006-12-16 17:24:00            18.400           0.000           1.000   
2006-12-16 17:25:00            23.000           0.000           1.000   
2006-12-16 17:26:00            23.000           0.000           2.000   
2006-12-16 17:27:00            23.000           0.000           1.000   
2006-12-16 17:28:00          

In [ ]:
print(data)

                     Global_active_power  Global_reactive_power  Voltage  \
Date_time                                                                  
2006-12-16 17:24:00                4.216                  0.418  234.840   
2006-12-16 17:25:00                5.360                  0.436  233.630   
2006-12-16 17:26:00                5.374                  0.498  233.290   
2006-12-16 17:27:00                5.388                  0.502  233.740   
2006-12-16 17:28:00                3.666                  0.528  235.680   
...                                  ...                    ...      ...   
2010-11-26 20:58:00                0.946                  0.000  240.430   
2010-11-26 20:59:00                0.944                  0.000  240.000   
2010-11-26 21:00:00                0.938                  0.000  239.820   
2010-11-26 21:01:00                0.934                  0.000  239.700   
2010-11-26 21:02:00                0.932                  0.000  239.550   

           

In [ ]:
data.columns

Index(['Global_active_power', 'Global_reactive_power', 'Voltage',
       'Global_intensity', 'Sub_metering_1', 'Sub_metering_2',
       'Sub_metering_3', 'Year', 'Month', 'Day', 'Hour', 'DayOfWeek',
       'Weekend'],
      dtype='object')

### Dealing with missing values 'nan' with a test statistic

In [ ]:
# finding all columns that have 'nan'
droping_list_all=[]
for i in range(0,7):
    if not data.iloc[:, i].notnull().all():
        droping_list_all.append(i)
        #print(data.iloc[:,i].unique())
droping_list_all

[0, 1, 2, 3, 4, 5, 6]

In [ ]:
# filling nan with mean in any columns
for i in range(0,7):
        data.iloc[:,i]=data.iloc[:,i].fillna(data.iloc[:,i].mean())

In [ ]:
# Check to make sure that there are not more any nan
data.isnull().sum()

Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
Year                     0
Month                    0
Day                      0
Hour                     0
DayOfWeek                0
Weekend                  0
dtype: int64

## Exploratory Data Analysis and Visualizations

### Plotting features over time (Daily)

In [ ]:
# !pip install pandas numpy plotly
import pandas as pd
import plotly.express as px

In [ ]:
# # Sample data creation based on the provided structure (for demonstration purposes)
# dates = pd.date_range(start='2006-12-16 17:24:00', end='2010-11-26 21:02:00', freq='T')
# data = pd.DataFrame({
#     'Global_active_power': np.random.rand(len(dates)) * 5,
#     'Global_reactive_power': np.random.rand(len(dates)) * 2,
#     'Voltage': np.random.rand(len(dates)) * 240,
#     'Global_intensity': np.random.rand(len(dates)) * 20,
#     'Sub_metering_1': np.random.rand(len(dates)) * 1,
#     'Sub_metering_2': np.random.rand(len(dates)) * 1,
#     'Sub_metering_3': np.random.rand(len(dates)) * 1
# }, index=dates)

In [ ]:
# Resampling to daily data for plotting (optional, for better visualization)
data_resampled = data.resample('D').mean()

# List of columns to plot
columns_to_plot = [
    'Global_active_power',
    'Global_reactive_power',
    'Voltage',
    'Global_intensity',
    'Sub_metering_1',
    'Sub_metering_2',
    'Sub_metering_3'
]

In [ ]:
# Function to plot each column
def plot_column(data, column_name):
    fig = px.line(data, x=data.index, y=column_name, title=f'{column_name.replace("_", " ").title()} Over Time (Daily)')
    fig.update_layout(
        plot_bgcolor='rgb(35, 35, 35)',     # Dark gray plot background
        paper_bgcolor='rgb(25, 25, 25)',    # Very dark gray background
        font=dict(color='lightgray'),       # Light gray font color
        title_x=0.5,                        # Title center alignment
        xaxis_title="Date",                 # X-axis title
        yaxis_title=column_name.replace("_", " ").title()  # Y-axis title
    )
    fig.show()

# Plot each column for the resampled data
for column in columns_to_plot:
    plot_column(data_resampled, column)

### Plotting Seasonal Decompositions

#### Plotting Additive Seasonal Decompositions with plotly

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
import plotly.graph_objects as go
import plotly.subplots as sp

# Perform seasonal decomposition
result = seasonal_decompose(data['Global_active_power'], model='additive', period=12)

# Plot the decomposed components
fig = sp.make_subplots(rows=4, cols=1, shared_xaxes=True,
                       subplot_titles=['Observed', 'Trend', 'Seasonal', 'Residual'])

# Add traces for each component
fig.add_trace(go.Scatter(x=data.index, y=result.observed, mode='lines', name='Observed'), row=1, col=1)
fig.add_trace(go.Scatter(x=data.index, y=result.trend, mode='lines', name='Trend'), row=2, col=1)
fig.add_trace(go.Scatter(x=data.index, y=result.seasonal, mode='lines', name='Seasonal'), row=3, col=1)
fig.add_trace(go.Scatter(x=data.index, y=result.resid, mode='lines', name='Residual'), row=4, col=1)

# Customize layout
fig.update_layout(
    plot_bgcolor='rgb(35, 35, 35)',    # Dark gray plot background
    paper_bgcolor='rgb(25, 25, 25)',    # Very dark gray background
    font=dict(color='lightgray'),      # Light gray font color
    height=1000,                         # Set figure height
    title='Additive Seasonal Decomposition of Global_active_power'  # Figure title
)

# Update subplot titles
for i in range(1, 5):
    fig.update_yaxes(title_text=result.observed.name if i == 1 else '', row=i, col=1)

fig.show()

#### Plotting Multiplicative Seasonal Decomposition plotly

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
import plotly.graph_objects as go
import plotly.subplots as sp

# Perform seasonal decomposition
result = seasonal_decompose(data['Global_active_power'], model='multiplicative', period=12)

# Plot the decomposed components
fig = sp.make_subplots(rows=4, cols=1, shared_xaxes=True,
                       subplot_titles=['Observed', 'Trend', 'Seasonal', 'Residual'])

# Add traces for each component
fig.add_trace(go.Scatter(x=data.index, y=result.observed, mode='lines', name='Observed'), row=1, col=1)
fig.add_trace(go.Scatter(x=data.index, y=result.trend, mode='lines', name='Trend'), row=2, col=1)
fig.add_trace(go.Scatter(x=data.index, y=result.seasonal, mode='lines', name='Seasonal'), row=3, col=1)
fig.add_trace(go.Scatter(x=data.index, y=result.resid, mode='lines', name='Residual'), row=4, col=1)

# Customize layout
fig.update_layout(
    plot_bgcolor='rgb(35, 35, 35)',    # Dark gray plot background
    paper_bgcolor='rgb(25, 25, 25)',    # Very dark gray background
    font=dict(color='lightgray'),      # Light gray font color
    height=1000,                         # Set figure height
    title='Additive Seasonal Decomposition of Global_active_power'  # Figure title
)

# Update subplot titles
for i in range(1, 5):
    fig.update_yaxes(title_text=result.observed.name if i == 1 else '', row=i, col=1)

fig.show()

### Plotting All Features over time (Monthly)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set figure size and style
plt.figure(figsize=(12, 15))
sns.set_style('darkgrid')

# Plotting features over time (month)
plt.subplot(7, 1, 1)
sns.lineplot(x='Month', y='Global_active_power', data=data, color='purple')
plt.title('Global_active_power Over Month')

plt.subplot(7, 1, 2)
sns.lineplot(x='Month', y='Global_reactive_power', data=data, color='darkorange')
plt.title('Global_reactive_power Over Month')

plt.subplot(7, 1, 3)
sns.lineplot(x='Month', y='Voltage', data=data, color='green')
plt.title('Voltage Over Month')

plt.subplot(7, 1, 4)
sns.lineplot(x='Month', y='Global_intensity', data=data, color='darkcyan')
plt.title('Global_intensity Over Month')


plt.subplot(7, 1, 5)
sns.lineplot(x='Month', y='Sub_metering_1', data=data, color='darkcyan')
plt.title('Sub_metering_1 Over Month')

plt.subplot(7, 1, 6)
sns.lineplot(x='Month', y='Sub_metering_2', data=data, color='darkcyan')
plt.title('Sub_metering_2 Over Month')

plt.subplot(7, 1, 7)
sns.lineplot(x='Month', y='Sub_metering_3', data=data, color='darkcyan')
plt.title('Sub_metering_ Over Month')

# Adjust layout
plt.tight_layout()
plt.show()

### Plotting Global_active_power over the years (by month)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set figure size and style
plt.figure(figsize=(20, 8))
sns.set_style('darkgrid')

# Plot Global_active_power over the years by month
plt.title("Global_active_power Over the Years")
sns.lineplot(data=data, x='Month', y='Global_active_power', hue='Year', palette='plasma', ci=None)

# Add labels
plt.xlabel("Month")
plt.ylabel("Global_active_power")

# Show plot
plt.show()

### Monthly Average (Global_active_power, Global_reactive_power, Voltage, Global_intensity, Sub_metering_1, Sub_metering_2, Sub_metering_3) Heatmap with Annotations for each year

In [ ]:
import plotly.graph_objects as go

def create_heatmap_trace(data, feature):
    data_filled = data.fillna(method='ffill')  # Forward fill missing values
    heatmap_data = data_filled.pivot_table(values=feature, index='Year', columns='Month', aggfunc='mean')
    heatmap_text = heatmap_data.round(2).astype(str).values

    heatmap = go.Heatmap(
        z=heatmap_data.values,
        x=heatmap_data.columns,
        y=heatmap_data.index,
        colorscale='thermal',
        text=heatmap_text,
        hoverinfo='text'
    )

    annotations = []
    for i in range(len(heatmap_data.index)):
        for j in range(len(heatmap_data.columns)):
            annotations.append(
                go.layout.Annotation(
                    x=heatmap_data.columns[j],
                    y=heatmap_data.index[i],
                    text=heatmap_text[i][j],
                    showarrow=False,
                    font=dict(color='white' if heatmap_data.values[i, j] < (heatmap_data.values.max() / 2) else 'black')
                )
            )

    return heatmap, annotations

In [ ]:
# Create heatmap traces and annotations for each feature
features = ['Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']
titles = ['Monthly Average Global_active_power', 'Monthly Average Global_reactive_power', 'Monthly Average Voltage', 'Monthly Average Global_intensity', 'Monthly Average Sub_metering_1', 'Monthly Average Sub_metering_2', 'Monthly Average Sub_metering_3']

# Create empty lists to store heatmap traces and annotations
heatmap_traces = []
annotations_list = []

# Iterate over each feature to create heatmap traces and annotations
for feature, title in zip(features, titles):
    heatmap, annotations = create_heatmap_trace(data, feature)
    heatmap_traces.append(heatmap)
    annotations_list.append(annotations)

In [ ]:
# Initialize figure with all traces but only show the first one
fig = go.Figure(data=heatmap_traces)

# Set initial visibility
for i, trace in enumerate(fig.data):
    trace.visible = (i == 0)

# Set layout properties
fig.update_layout(
    title=titles[0],
    xaxis=dict(nticks=12, title='Month'),
    yaxis=dict(title='Year'),
    annotations=annotations_list[0],
    updatemenus=[
        dict(
            buttons=[
                dict(
                    args=[{'visible': [j == i for j in range(len(features))]},
                          {'annotations': annotations_list[i],
                           'title': titles[i]}],
                    label=titles[i],
                    method='update'
                )
                for i in range(len(features))
            ],
            direction='down',
            showactive=True,
            x=1.15,  # Positioning the button to the right
            y=1.15   # Positioning the button at the top
        )
    ]
)

fig.show()

### Heatmap with seaborn

In [ ]:
# Set up the figure size
plt.figure(figsize=(10, 8))

# Compute the correlation matrix for numeric columns
correlation_matrix = df.select_dtypes('number').corr()

# Customize the heatmap
sns.heatmap(correlation_matrix, cmap='coolwarm', annot=True, fmt='.2f',
            linewidths=0.5, annot_kws={"size": 10}, cbar_kws={"shrink": 0.8})

# Add title
plt.title('Correlation Matrix of Numeric Features', fontsize=16)

# Adjust tick labels rotation for better readability
plt.xticks(rotation=45)
plt.yticks(rotation=0)

# Show plot
plt.tight_layout()
plt.show()

### Correlation Barplot with Global_active_power feature

In [ ]:
# Set up the figure size and style
plt.figure(figsize=(12, 6))
sns.set_style('whitegrid')

# Compute the correlation matrix for numeric columns
correlation_matrix = round(df.select_dtypes('number').corr(), 2)

# Sort correlations with meantemp in descending order
correlation_with_trgt = correlation_matrix['Global_active_power'].sort_values(ascending=False)

# Create the bar plot
ax = sns.barplot(x=correlation_with_trgt.index, y=correlation_with_trgt, palette='coolwarm')

# Title and axis labels
plt.title('Correlation with Mean Temperature', size=16)
plt.xlabel('Features', size=12)
plt.ylabel('Correlation', size=12)

# Add value annotations on top of bars
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 5), textcoords='offset points', fontsize=10)

# Rotate x-axis labels for better readability
plt.xticks(rotation=45, ha='right')

# Show plot
plt.tight_layout()
plt.show()

### Monthly Distributions of features with Box Plots

In [ ]:
import plotly.express as px

# Create subplots for each feature
fig = px.box(data, x='Month', y='Global_active_power', title='Monthly Distribution of Mean Global_active_power', template='plotly_dark')
fig.update_xaxes(title_text='Month')
fig.update_yaxes(title_text='Mean Global_active_power')

fig2 = px.box(data, x='Month', y='Global_reactive_power', title='Monthly Distribution of Mean Global_reactive_power', template='plotly_dark')
fig2.update_xaxes(title_text='Month')
fig2.update_yaxes(title_text='Mean Global_reactive_power')

fig3 = px.box(data, x='Month', y='Voltage', title='Monthly Distribution of Mean Voltage', template='plotly_dark')
fig3.update_xaxes(title_text='Month')
fig3.update_yaxes(title_text='Mean Voltage')

fig4 = px.box(data, x='Month', y='Global_intensity', title='Monthly Distribution of Mean Global_intensity', template='plotly_dark')
fig4.update_xaxes(title_text='Month')
fig4.update_yaxes(title_text='Mean Global_intensity')

fig5 = px.box(data, x='Month', y='Sub_metering_1', title='Monthly Distribution of Mean Sub_metering_1', template='plotly_dark')
fig5.update_xaxes(title_text='Month')
fig5.update_yaxes(title_text='Mean Sub_metering_1')

fig6 = px.box(data, x='Month', y='Sub_metering_2', title='Monthly Distribution of Mean Sub_metering_2', template='plotly_dark')
fig6.update_xaxes(title_text='Month')
fig6.update_yaxes(title_text='Mean Sub_metering_2')

fig7 = px.box(data, x='Month', y='Sub_metering_3', title='Monthly Distribution of Mean Sub_metering_3', template='plotly_dark')
fig7.update_xaxes(title_text='Month')
fig7.update_yaxes(title_text='Mean Sub_metering_3')

# Arrange subplots in a grid
fig.update_layout(
    grid={'rows': 2, 'columns': 2, 'pattern': "independent"},
)

# Show plots
fig.show()
fig2.show()
fig3.show()
fig4.show()
fig5.show()
fig6.show()
fig7.show()

### Scatterplots

In [ ]:
import plotly.express as px

# Create scatter matrix plot
fig = px.scatter_matrix(data,
                        dimensions=['Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3', 'Year', 'Month', 'Day', 'Hour', 'DayOfWeek', 'Weekend'],
                        title='Scatter Matrix of Weather Parameters')

# Update layout to increase figure size and add outlines to dots
fig.update_layout(
    width=1500,  # Increased width of the figure
    height=1000,  # Increased height of the figure
    title_x=0.5,  # Title position
    margin=dict(l=50, r=50, t=50, b=50),  # Margin around the plot
    template='plotly_dark'
)

# Update marker properties to add outlines
fig.update_traces(marker=dict(line=dict(width=1, color='black')))  # Add outlines to dots

# Show the figure
fig.show()

### Histograms

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

# List of features to create histograms for
features = ['Global_active_power', 'Global_reactive_power', 'Voltage',
            'Global_intensity', 'Sub_metering_1', 'Sub_metering_2',
            'Sub_metering_3']

# Create histograms for each feature
histograms = []
for feature in features:
    fig = px.histogram(df, x=feature, marginal='rug', title=f'Histogram for {feature}')
    fig.update_traces(marker=dict(line=dict(width=1, color='black')))  # Add outlines to bars
    histograms.append(fig)

# Create subplot layout
fig = go.Figure()

# Add traces to the subplot
for histogram in histograms:
    fig.add_trace(histogram['data'][0])

# Define button list for toggling between plots
buttons = []
for i, feature in enumerate(features):
    button = dict(label=feature, method='update', args=[{'visible': [idx == i for idx in range(len(features))]}])
    buttons.append(button)

# Add buttons to the figure
fig.update_layout(
    updatemenus=[dict(buttons=buttons, direction='down', showactive=True, x=1.0, y=1.15)],
    title='Histograms with Outlines',
    template='plotly_dark'
)

# Show the figure
fig.show()


### Histograms with Seaborn


In [ ]:
# Reset seaborn style to default
sns.set_style('darkgrid')

# Create subplots
fig, ax = plt.subplots(4, 3, figsize=(20, 20))

# Plot histograms for each feature
for i, feature in enumerate(['Global_active_power', 'Global_reactive_power', 'Voltage',
                             'Global_intensity', 'Sub_metering_1', 'Sub_metering_2',
                             'Sub_metering_3']):
    row = i // 3
    col = i % 3
    sns.histplot(data=df, x=feature, ax=ax[row][col], color='skyblue', kde=True)
    ax[row][col].set_title(f'Histogram for {feature}')

# Remove empty subplots
for i in range(1):
    for j in range(3):
        ax[3][j].remove()

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()

### Polar Plots

In [ ]:
data.columns

In [ ]:
# Reset seaborn style to default
sns.set_style('darkgrid')

# Create a 4x2 grid of subplots
fig, axs = plt.subplots(4, 2, subplot_kw={'projection': 'polar'}, figsize=(14, 20))

# List of parameters and their titles
parameters = ['Global_active_power', 'Global_reactive_power', 'Voltage',
       'Global_intensity', 'Sub_metering_1', 'Sub_metering_2',
       'Sub_metering_3']
# Titles for each parameter
titles = ['Global Active Power', 'Global Reactive Power', 'Voltage',
          'Global Intensity', 'Sub Metering 1', 'Sub Metering 2',
          'Sub Metering 3']

# Define a color palette for better distinction between parameters
# colors = ['skyblue', 'lightgreen', 'lightcoral', 'lightcoral',
#           'lightskyblue', 'lightgreen', 'lightcoral']

for ax, param, title in zip(axs.flatten(), parameters, titles):
    # Grouping the data by month, calculating the average mean value for each month
    monthly_average = data.groupby('Month')[param].mean()

    # Polar Plot theta (angle) and radii (length) settings
    theta = np.linspace(0, 2 * np.pi, len(monthly_average), endpoint=False)
    radii = monthly_average.values

    # Extend theta and radii to connect the circle
    theta = np.append(theta, theta[0])
    radii = np.append(radii, radii[0])

    # Polar Plot
    ax.plot(theta, radii, label=title)
    ax.set_title(title, va='bottom')
    ax.set_xticks(theta[:-1])
    ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May',
                        'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
    ax.set_ylim(0, radii.max() + 10)
    ax.legend(loc='upper right')

plt.tight_layout()
plt.show()

## Feature Selection

> Average Voltage is almost the same in every month and has almost no correlation with Global Active Power

In [ ]:
# Selecting the desired features for modeling
data = data[['Global_active_power', 'Global_reactive_power', 'Voltage',
            'Global_intensity', 'Sub_metering_1', 'Sub_metering_2',
            'Sub_metering_3']]

# Displaying the first few rows of the updated DataFrame
print(data.head())

                     Global_active_power  Global_reactive_power  Voltage  \
Date_time                                                                  
2006-12-16 17:24:00                4.216                  0.418  234.840   
2006-12-16 17:25:00                5.360                  0.436  233.630   
2006-12-16 17:26:00                5.374                  0.498  233.290   
2006-12-16 17:27:00                5.388                  0.502  233.740   
2006-12-16 17:28:00                3.666                  0.528  235.680   

                     Global_intensity  Sub_metering_1  Sub_metering_2  \
Date_time                                                               
2006-12-16 17:24:00            18.400           0.000           1.000   
2006-12-16 17:25:00            23.000           0.000           1.000   
2006-12-16 17:26:00            23.000           0.000           2.000   
2006-12-16 17:27:00            23.000           0.000           1.000   
2006-12-16 17:28:00          

In [ ]:
# Display information about the DataFrame
print(data.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2075259 entries, 2006-12-16 17:24:00 to 2010-11-26 21:02:00
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Global_active_power    float64
 1   Global_reactive_power  float64
 2   Voltage                float64
 3   Global_intensity       float64
 4   Sub_metering_1         float64
 5   Sub_metering_2         float64
 6   Sub_metering_3         float64
dtypes: float64(7)
memory usage: 126.7 MB
None


## Machine Learning Models

### ARIMA-SARIMA Models

#### Data Preparation

In [ ]:
# Creating a copy of the DataFrame
data1 = data.copy()

# Displaying the new DataFrame
data1.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
Date_time,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.000
2006-12-16 17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.000
2006-12-16 17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.000
2006-12-16 17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.000
2006-12-16 17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.000


#### Check Stationarity

##### ACF (Autocorrelation Function) & PACF (Partial Autocorrelation Function)

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot ACF
plot_acf(data1['Global_active_power'], ax=axes[0], lags=40, title='Autocorrelation Function (ACF) for Global_active_power')

# Plot PACF
plot_pacf(data1['Global_active_power'], ax=axes[1], lags=40, title='Partial Autocorrelation Function (PACF) for Global_active_power')

# Add some additional customization
for ax in axes:
    ax.tick_params(axis='x', labelsize=10)  # Adjust x-axis tick labels size
    ax.tick_params(axis='y', labelsize=10)  # Adjust y-axis tick labels size
    ax.grid(True)  # Add gridlines for better readability

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()

> The Series does not seem stationary, ACF has to become 0 at some point. But we are not certainly sure yet.

Let's confirm with the ADF and KPSS tests

##### Augmented Dickey-Fuller Test (ADF) & Kwiatkowski-Phillips-Schmidt-Shin Test (KPSS)

In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, kpss

def check_stationarity(series, alpha=0.05):
    print(f'\n___________________Checking Stationarity for: {series.name}___________________\n')

    # Handle missing values
    series = series.dropna()

    # ADF Test
    adf_test = adfuller(series.values)
    print('ADF Test:')
    print(f'ADF Statistic: {adf_test[0]}')
    print(f'p-value: {adf_test[1]}')
    print('Critical Values:')
    for key, value in adf_test[4].items():
        print(f'\t{key}: {value:.3f}')
    if adf_test[1] <= alpha and adf_test[4]['5%'] > adf_test[0]:
        print("\x1b[32mSeries is Stationary (ADF Test)\x1b[0m")
    else:
        print("\x1b[31mSeries is Non-stationary (ADF Test)\x1b[0m")

    print('-' * 50)

    # KPSS Test
    kpss_test = kpss(series.values, regression='c', nlags='auto')
    print('KPSS Test:')
    print(f'KPSS Statistic: {kpss_test[0]}')
    print(f'p-value: {kpss_test[1]}')
    print('Critical Values:')
    for key, value in kpss_test[3].items():
        print(f'\t{key}: {value:.3f}')
    if kpss_test[1] > alpha:
        print("\x1b[32mSeries is Stationary (KPSS Test)\x1b[0m")
    else:
        print("\x1b[31mSeries is Non-stationary (KPSS Test)\x1b[0m")

In [ ]:
# Check initial stationarity for each feature
check_stationarity(data1['Global_active_power'])  # Target is non-stationary!!
check_stationarity(data1['Global_reactive_power'])
check_stationarity(data1['Global_intensity'])
check_stationarity(data1['Sub_metering_1'])
check_stationarity(data1['Sub_metering_2'])
check_stationarity(data1['Sub_metering_3'])

##### Apply differencing to make the series stationary

In [ ]:
def check_stationarity_after_diff(series, diff_degree=1, alpha=0.05, plot=True):
    print('\n\n############################### After Differencing ###############################\n\n')

    # Apply differencing
    series_diff = series.diff(diff_degree).fillna(0)

    # Check stationarity after differencing
    check_stationarity(series_diff, alpha=alpha)

    # Optionally plot original and differenced series
    if plot:
        plt.figure(figsize=(12, 6))
        plt.plot(series, label='Original Series')
        plt.plot(series_diff, label=f'Differenced Series (Degree={diff_degree})')
        plt.title('Original vs. Differenced Time Series')
        plt.xlabel('Index')
        plt.ylabel('Value')
        plt.legend()
        plt.show()

# Check initial stationarity for 'Global_active_power' series
check_stationarity(data1['Global_active_power'])

# Check stationarity after differencing for 'Global_active_power' series
check_stationarity_after_diff(data1['Global_active_power'])


___________________Checking Stationarity for: Global_active_power___________________



##### Plotting ACF and PACF before and after differencing

In [ ]:
# Create 'Global_active_power_diff' column by differencing the 'Global_active_power' series
data1['Global_active_power_diff'] = data1['Global_active_power'].diff().fillna(0)

# Create subplots
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(15, 8))

# Plot ACF and PACF for non-stationary 'Global_active_power' series
plot_acf(data1['Global_active_power'], lags=40, ax=ax[0, 0])
ax[0, 0].set_title('ACF on Non-Stationary (Original)')
plot_pacf(data1['Global_active_power'], lags=40, ax=ax[0, 1], method='ols')
ax[0, 1].set_title('PACF on Non-Stationary (Original)')

# Plot ACF and PACF for differenced/stationary 'Global_active_power' series
plot_acf(data1['Global_active_power_diff'], lags=40, ax=ax[1, 0])
ax[1, 0].set_title('ACF on Differenced/Stationary')
plot_pacf(data1['Global_active_power_diff'], lags=40, ax=ax[1, 1], method='ols')
ax[1, 1].set_title('PACF on Differenced/Stationary')

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()

> After differencing, the series has become stationary

#### Spliting data for ARIMA-SARIMA Model

In [ ]:
# Check DataFrame information
data1.info()

In [ ]:
# Split the data into training and testing sets
train_size = int(len(data1) * 0.8)  # 80% training data, 20% testing data
train, test = data1.iloc[:train_size], data1.iloc[train_size:]

# Display training set information
print(f'Train shape: {train.shape}')

# Display testing set information
print(f'Test shape: {test.shape}')

In [ ]:
print(train.head())

In [ ]:
print(test.head())

#### ARIMA modeling process with Daily data

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Fit the ARIMA model
arima_model = ARIMA(train['Global_active_power'], order=(1,1,1))  # ARIMA(p,d,q)
arima_model_fit = arima_model.fit()

In [ ]:
# Make predictions
arima_pred = arima_model_fit.forecast(steps=len(test))

In [ ]:
# Calculate error metrics
mse = mean_squared_error(test['Global_active_power'], arima_pred)
mae = mean_absolute_error(test['Global_active_power'], arima_pred)
print('Test MSE: %.3f' % mse)
print('Test MAE: %.3f' % mae)

In [ ]:
# Plot the entire time series with forecast
plt.figure(figsize=(12, 6))
plt.plot(data1.index, data1['Global_active_power'], label='Actual')
plt.plot(test.index, arima_pred, color='red', label='Forecast')
plt.xlabel('Date')
plt.ylabel('Mean Temperature')
plt.title('ARIMA Forecast')
plt.legend()
plt.show()

> Simple ARIMA Model is not suitable for this data (There is seasonality in our data)

#### SARIMAX modeling process Daily data

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Define the SARIMA model parameters
order = (1, 1, 6)  # Non-seasonal order (p, d, q)
seasonal_order = (1, 1, 1, 7)  # Seasonal order (P, D, Q, S)

# Fit the SARIMA model
sarima_model = SARIMAX(endog=train['Global_active_power'], exog=train[['Global_reactive_power', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']],
                       order=order, seasonal_order=seasonal_order)

sarima_model_fit = sarima_model.fit()

In [ ]:
# Make predictions
sarima_pred = sarima_model_fit.predict(start=test.index[0], end=test.index[-1],
                                        exog=test[['Global_reactive_power', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']])

In [ ]:
# Calculate error metrics
mse = mean_squared_error(test['Global_active_power'], sarima_pred)
mae = mean_absolute_error(test['Global_active_power'], sarima_pred)
r2 = r2_score(test['Global_active_power'], sarima_pred)
print('Test MSE:', mse)
print('Test MAE:', mae)
print('Test R²: %.3f' % r2)

In [ ]:
# Plot the entire time series with forecast
plt.figure(figsize=(12, 6))
plt.plot(data1.index, data1['Global_active_power'], label='Actual')
plt.plot(test.index, sarima_pred, color='red', label='SARIMA Forecast')
plt.xlabel('Date')
plt.ylabel('Global_active_power')
plt.title('SARIMA Forecast')
plt.legend()
plt.show()

#### SARIMAX model with a differenced target

In [ ]:
# Fit the SARIMA model with differenced target
sarima_model = SARIMAX(endog=train['Global_active_power_diff'], exog=train[['Global_reactive_power', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']],
                       order=order, seasonal_order=seasonal_order)

sarima_model_fit = sarima_model.fit()

In [ ]:
# Make predictions
sarima_pred_diff = sarima_model_fit.predict(start=test.index[0], end=test.index[-1],
                                            exog=test[['Global_reactive_power', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']])

In [ ]:
# Convert differenced predictions back to the original scale
last_original_value = train['Global_active_power'].iloc[-1]
sarima_pred = sarima_pred_diff.cumsum() + last_original_value

In [ ]:
# Calculate error metrics
mse = mean_squared_error(test['Global_active_power'], sarima_pred)
mae = mean_absolute_error(test['Global_active_power'], sarima_pred)
r2 = r2_score(test['Global_active_power'], sarima_pred)
print('Test MSE:', mse)
print('Test MAE:', mae)
print('Test R²: %.3f' % r2)

In [ ]:
# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(test.index, test['Global_active_power'], label='Actual')
plt.plot(test.index, sarima_pred, color='red', label='SARIMA Forecast')
plt.xlabel('Date')
plt.ylabel('Global_active_power')
plt.title('SARIMA Forecast with Differenced Target')
plt.legend()
plt.show()

#### ARIMA with Monthly data




In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Resample the data to monthly frequency
monthly_df = data1.resample('M').mean()

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
m_train, m_test = train_test_split(monthly_df, test_size=0.2, shuffle=False)

# Split the data into training and testing sets
# train_size = int(len(monthly_df) * 0.8)
# m_train = monthly_df.iloc[:train_size]
# m_test = monthly_df.iloc[train_size:]

In [ ]:
# Fit the ARIMA model
arima_model = ARIMA(m_train['Global_active_power'], order=(1, 0, 0))  # ARIMA(p,d,q)
arima_model_fit = arima_model.fit()

In [ ]:
# Make predictions
arima_pred_m = arima_model_fit.forecast(steps=len(m_test))

In [ ]:
# Calculate error metrics
mse = mean_squared_error(m_test['Global_active_power'], arima_pred_m)
mae = mean_absolute_error(m_test['Global_active_power'], arima_pred_m)
print('Test MSE: %.3f' % mse)
print('Test MAE: %.3f' % mae)

In [ ]:
# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(m_test.index, m_test['Global_active_power'], label='Actual')
plt.plot(m_test.index, arima_pred_m, color='red', label='Forecast')
plt.xlabel('Date')
plt.ylabel('Global_active_power')
plt.title('ARIMA Forecast (Monthly Data)')
plt.legend()
plt.show()

#### SARIMA with Monthly data


In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Define the SARIMA model parameters
order = (1, 0, 0)  # Non-seasonal order (p, d, q)
seasonal_order = (2, 1, 1, 12)  # Seasonal order (P, D, Q, S)

# Fit the SARIMA model
sarima_model = SARIMAX(endog=m_train['Global_active_power'], exog=m_train[['Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']],
                       order=order, seasonal_order=seasonal_order)
sarima_model_fit = sarima_model.fit()

In [ ]:
# Make predictions
sarima_pred_m = sarima_model_fit.predict(start=m_test.index[0], end=m_test.index[-1],
                                          exog=m_test[['Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']])

In [ ]:
# Calculate error metrics
mse = mean_squared_error(m_test['Global_active_power'], sarima_pred_m)
mae = mean_absolute_error(m_test['Global_active_power'], sarima_pred_m)
r2 = r2_score(m_test['Global_active_power'], sarima_pred_m)
print('Test MSE:', mse)
print('Test MAE:', mae)
print('Test R²: %.3f' % r2)

In [ ]:
# Plot the results
plt.figure(figsize=(10, 5))
plt.plot(m_test.index, m_test['Global_active_power'], label='Actual')
plt.plot(m_test.index, sarima_pred_m, color='red', label='SARIMA Forecast')
plt.xlabel('Date')
plt.ylabel('Global_active_power')
plt.title('SARIMA Forecast')
plt.legend()
plt.show()

> I think figuring out the p, d, and q parameters are quite hard from ACF and PACF plots.

So we will try forecasting with auto_arima

#### Modeling with auto_arima (monthly prediction)

In [ ]:
!pip install pmdarima

In [ ]:
from pmdarima import auto_arima
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Fit the model on the training data with optimized settings for monthly seasonality
model = auto_arima(
    m_train['Global_active_power'],
    seasonal=True,
    m=12,
    max_p=7,
    max_d=1,
    max_q=3,
    max_P=3,
    max_D=1,
    max_Q=2,
    trace=True,    # To print the progress of the fitting
    error_action='ignore',  # Ignore potential errors
    suppress_warnings=True,  # Suppress warnings
    n_jobs=-1  # Use all available CPU cores
)

# Print model summary
print(model.summary())

In [ ]:
# Make predictions
n_periods = len(m_test)
auto_arima_pred = model.predict(n_periods=n_periods)

In [ ]:
# Calculate evaluation metrics
r2 = r2_score(m_test['Global_active_power'], auto_arima_pred)
mse = mean_squared_error(m_test['Global_active_power'], auto_arima_pred)
mae = mean_absolute_error(m_test['Global_active_power'], auto_arima_pred)

print(f'R² score: {r2}')
print(f'MSE: {mse}')
print(f'MAE: {mae}')

In [ ]:
# Plot the results
plt.figure(figsize=(14, 7))
plt.plot(m_train.index, m_train['Global_active_power'], label='Train')
plt.plot(m_test.index, m_test['Global_active_power'], label='Test')
plt.plot(m_test.index, auto_arima_pred, label='Predicted')
plt.legend()
plt.xlabel('Date')
plt.ylabel('Global_active_power')
plt.title('Global_active_power Prediction')
plt.show()

### Prophet Model

#### Prophet Model with only Global_reactive_power features

In [ ]:
data.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
Date_time,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.000
2006-12-16 17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.000
2006-12-16 17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.000
2006-12-16 17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.000
2006-12-16 17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.000


In [ ]:
data.columns

Index(['Global_active_power', 'Global_reactive_power', 'Voltage',
       'Global_intensity', 'Sub_metering_1', 'Sub_metering_2',
       'Sub_metering_3'],
      dtype='object')

In [ ]:
from prophet import Prophet

# Rename columns for Prophet
df_prophet = data.reset_index().rename(columns={'Date_time': 'ds', 'Global_active_power': 'y'})

# Split the data into training and testing sets
train_size = int(len(df_prophet) * 0.8)
p_train, p_test = df_prophet.iloc[:train_size], df_prophet.iloc[train_size:]

# Initialize and fit the Prophet model
prop_model = Prophet()
prop_model.fit(p_train)

# Make future dataframe
future = prop_model.make_future_dataframe(periods=len(p_test), freq='D')

DEBUG:cmdstanpy:input tempfile: /tmp/tmpg4vw095r/h7qg6o_s.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpg4vw095r/v_fujdqk.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=81146', 'data', 'file=/tmp/tmpg4vw095r/h7qg6o_s.json', 'init=/tmp/tmpg4vw095r/v_fujdqk.json', 'output', 'file=/tmp/tmpg4vw095r/prophet_modeleo9_6m82/prophet_model-20240605152409.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
15:24:09 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing


KeyboardInterrupt: 

In [ ]:
# Make predictions
forecast = prop_model.predict(future)

In [ ]:
# Extract the forecasted values for the test period
predicted = forecast[['ds', 'yhat']].set_index('ds').loc[p_test['ds']

In [ ]:
# Calculate error and R²
mse = mean_squared_error(p_test['y'], predicted['yhat'])
r2 = r2_score(p_test['y'], predicted['yhat'])
print('Test MSE: %.3f' % mse)
print('Test R²: %.3f' % r2)

In [ ]:
# Plot the results
plt.figure(figsize=(15, 5))
plt.plot(df_prophet['ds'], df_prophet['y'], label='Actual')
plt.plot(predicted.index, predicted['yhat'], color='red', label='Forecast')
plt.xlabel('Date')
plt.ylabel('Global_active_power')
plt.title('Prophet Forecast')
plt.legend()
plt.show()

#### Prophet Model using the entire dataset

In [ ]:
from prophet import Prophet

# Rename columns for Prophet
df_prophet = data.reset_index().rename(columns={'Date_time': 'ds', 'Global_active_power': 'y'})

# Add additional features as regressors
regressors = ['Global_reactive_power', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']
for regressor in regressors:
    df_prophet[regressor] = data[regressor].values

In [ ]:
# Split the data into training and testing sets
train_size = int(len(df_prophet) * 0.8)
p_train, p_test = df_prophet.iloc[:train_size], df_prophet.iloc[train_size:]

In [ ]:
# Initialize and fit the Prophet model
prop_model = Prophet()

# Initialize and fit the Prophet model
prop_model = Prophet()
for regressor in regressors:
    prop_model.add_regressor(regressor)


# Fit the model
prop_model.fit(p_train)

In [ ]:
# Make future dataframe
future = prop_model.make_future_dataframe(periods=len(p_test), freq='D')

# Add the regressors to the future dataframe
for regressor in regressors:
    future[regressor] = df_prophet[regressor]

# Make predictions
forecast = prop_model.predict(future)

# Extract the forecasted values for the test period
predicted = forecast[['ds', 'yhat']].set_index('ds').loc[p_test['ds']]

In [ ]:
# Calculate error and R²
mse = mean_squared_error(p_test['y'], predicted['yhat'])
r2 = r2_score(p_test['y'], predicted['yhat'])
print('Test MSE: %.3f' % mse)
print('Test R²: %.3f' % r2)

In [ ]:
# Plot the results
plt.figure(figsize=(15, 5))
plt.plot(df_prophet['ds'], df_prophet['y'], label='Actual')
plt.plot(predicted.index, predicted['yhat'], color='red', label='Forecast')
plt.xlabel('Date')
plt.ylabel('Global_active_power')
plt.title('Prophet Forecast')
plt.legend()
plt.show()

> The Prophet model achieved an R² score of ~91% when using the entire dataset with all features, indicating a high level of accuracy.

> In contrast, the R² score dropped to 76% when the model was trained without the additional features.

#### Working with holiday for Prophet Model using the entire dataset





In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from prophet import Prophet

# Prepare Data for Prophet
df_prophet = data.reset_index().rename(columns={'Date_time': 'ds', 'Global_active_power': 'y'})

# Add additional features as regressors
regressors = ['Global_reactive_power', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']
for regressor in regressors:
    df_prophet[regressor] = data[regressor].values

# Split the data into training and testing sets
train_size = int(len(df_prophet) * 0.8)
p_train, p_test = df_prophet.iloc[:train_size], df_prophet.iloc[train_size:]


In [ ]:
# Initialize and fit the Prophet model
prop_model = Prophet()
for regressor in regressors:
    prop_model.add_regressor(regressor)

# Add holidays
prop_model.add_country_holidays(country_name='US')

# Fit the model
prop_model.fit(p_train)

In [ ]:
# Make future dataframe
future = prop_model.make_future_dataframe(periods=len(p_test), freq='D')

# Add the regressors to the future dataframe
for regressor in regressors:
    future[regressor] = df_prophet[regressor]

In [ ]:
# Make predictions
forecast = prop_model.predict(future)

# Extract the forecasted values for the test period
predicted = forecast[['ds', 'yhat']].set_index('ds').loc[p_test['ds']]

In [ ]:
# Calculate error and R²
mse = mean_squared_error(p_test['y'], predicted['yhat'])
r2 = r2_score(p_test['y'], predicted['yhat'])
print('Test MSE: %.3f' % mse)
print('Test R²: %.3f' % r2)

In [ ]:
# Plot the results
plt.figure(figsize=(15, 5))
plt.plot(df_prophet['ds'], df_prophet['y'], label='Actual', color='blue')
plt.plot(predicted.index, predicted['yhat'], color='red', label='Forecast', alpha=0.7)
plt.fill_between(predicted.index, predicted['yhat_lower'], predicted['yhat_upper'], color='k', alpha=0.1, label='Confidence Interval')
plt.xlabel('Date')
plt.ylabel('Global Active Power')
plt.title('Prophet Forecast with Holidays and Regressors')
plt.legend()
plt.show()

In [ ]:
# Plot the results
plt.figure(figsize=(10, 5))
plt.plot(p_test['ds'], p_test['y'], label='Actual')
plt.plot(predicted.index, predicted['yhat'], color='red', label='Forecast', linestyle='--')
plt.xlabel('Date')
plt.ylabel('Global_active_power')
plt.title('Prophet Forecast')
plt.legend()

# Add error metrics to the plot
plt.text(0.05, 0.95, f'Test MSE: {mse:.3f}', transform=plt.gca().transAxes, fontsize=10,
         verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
plt.text(0.05, 0.9, f'Test R²: {r2:.3f}', transform=plt.gca().transAxes, fontsize=10,
         verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.show()

In [ ]:
print(df_prophet.head()) # actual data

print(predicted.head()) # predicted values of p_test

In [ ]:
forecast.head()

In [ ]:
# Adding the actual temperature values to the forecast data
forecast['Actual Global_active_power'] = df_prophet['y']
forecast.rename(columns={'yhat': 'Forecast Global_active_power'}, inplace=False)
forecast.head()

In [ ]:
plt.figure(figsize=(22, 8))
sns.lineplot(data=forecast[['ds', 'Actual Global_active_power', 'yhat_lower', 'yhat_upper']])
plt.xlabel('Date')
plt.ylabel('Global_active_power')
plt.title('Actual vs Forecast Global_active_power with Confidence Intervals')
plt.show()

In [ ]:
from prophet.plot import plot_plotly
import plotly.offline as py

# Plot the forecast
fig = plot_plotly(prop_model, forecast)

# Customize the layout
fig.update_layout(
    title="Prophet Forecast with Actual vs Predicted Global_active_power",
    xaxis_title="Date",
    yaxis_title="Global_active_power",
    legend_title="",
)

# Show the plot
py.iplot(fig)

In [ ]:
from prophet.plot import add_changepoints_to_plot
import matplotlib.pyplot as plt

# Plot the components of the forecast
fig = prop_model.plot_components(forecast)

# Customize the plots
axes = fig.get_axes()

# Trend plot
axes[0].set_title('Trend')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Trend')

# Weekly seasonality plot
axes[1].set_title('Weekly Seasonality')
axes[1].set_xlabel('Day of Week')
axes[1].set_ylabel('Seasonal Effect')

# Yearly seasonality plot
axes[2].set_title('Yearly Seasonality')
axes[2].set_xlabel('Day of Year')
axes[2].set_ylabel('Seasonal Effect')

# Add changepoints to the trend plot (if any)
fig = add_changepoints_to_plot(fig.gca(), prop_model, forecast)

# Display the plot
plt.tight_layout()
plt.show()

#### Comparison of ARIMA, SARIMA & Prophet Model

In [ ]:
# Predictions of ARIMA
print(arima_pred)

In [ ]:
# Predictions of SARIMA
print(sarima_pred)

In [ ]:
# predictions of Prophet
print(predicted)

In [ ]:
# Copying test data from arima/sarima models (for daily comparision)
pred_df = test.copy()
print(pred_df)

In [ ]:
# Display the first few rows of the monthly test data
print(m_test.head())

# Adding the predictions from ARIMA, SARIMA, and Auto ARIMA models to the test data
m_test["arima_pred_m"] = arima_pred_m.round(2)
m_test["sarima_pred_m"] = sarima_pred_m.round(2)
m_test["auto_arima_pred"] = auto_arima_pred.round(2)

# Display the first few rows of the updated test data with predictions
print(m_test.head())

In [ ]:
# Adding predictions of ARIMA, SARIMA, and Prophet to the DataFrame
pred_df["arima_pred"] = arima_pred.values.round(2)
pred_df["sarima_pred"] = sarima_pred.values.round(2)
pred_df["prophet_yhat"] = predicted["yhat"].values.round(2)

# Display the first few rows of the DataFrame with predictions
print(pred_df)

#### Comparison of Forecasting Models (Prophet, ARIMA, and SARIMA on Daily and Monthly)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score
import math

# Prophet model evaluation metrics
prophet_mape = mean_absolute_percentage_error(pred_df["Global_active_power"].values, pred_df["prophet_yhat"].values)
prophet_r = r2_score(pred_df["Global_active_power"].values, pred_df["prophet_yhat"].values)
prophet_rmse = math.sqrt(mean_squared_error(pred_df["Global_active_power"].values, pred_df["prophet_yhat"].values))

# ARIMA model evaluation metrics
arima_mape = mean_absolute_percentage_error(pred_df["Global_active_power"].values, pred_df["arima_pred"].values)
arima_r = r2_score(pred_df["Global_active_power"].values, pred_df["arima_pred"].values)
arima_rmse = math.sqrt(mean_squared_error(pred_df["Global_active_power"].values, pred_df["arima_pred"].values))

# SARIMA model evaluation metrics
sarima_mape = mean_absolute_percentage_error(pred_df["Global_active_power"].values, pred_df["sarima_pred"].values)
sarima_r = r2_score(pred_df["Global_active_power"].values, pred_df["sarima_pred"].values)
sarima_rmse = math.sqrt(mean_squared_error(pred_df["Global_active_power"].values, pred_df["sarima_pred"].values))


In [ ]:
# Monthly predictions evaluation metrics
arima_mape_m = mean_absolute_percentage_error(m_test["Global_active_power"].values, m_test["arima_pred_m"].values)
arima_r_m = r2_score(m_test["Global_active_power"].values, m_test["arima_pred_m"].values)
arima_rmse_m = math.sqrt(mean_squared_error(m_test["Global_active_power"].values, m_test["arima_pred_m"].values))

sarima_mape_m = mean_absolute_percentage_error(m_test["Global_active_power"].values, m_test["sarima_pred_m"].values)
sarima_r_m = r2_score(m_test["Global_active_power"].values, m_test["sarima_pred_m"].values)
sarima_rmse_m = math.sqrt(mean_squared_error(m_test["Global_active_power"].values, m_test["sarima_pred_m"].values))

autoarima_mape_m = mean_absolute_percentage_error(m_test["Global_active_power"].values, m_test["auto_arima_pred"].values)
autoarima_r_m = r2_score(m_test["Global_active_power"].values, m_test["auto_arima_pred"].values)
autoarima_rmse_m = math.sqrt(mean_squared_error(m_test["Global_active_power"].values, m_test["auto_arima_pred"].values))


In [ ]:
# Creating a DataFrame to compare the metrics
compare_df = {
    'Prophet daily': [prophet_mape, prophet_rmse, prophet_r],
    'ARIMA daily': [arima_mape, arima_rmse, arima_r],
    'SARIMA daily': [sarima_mape, sarima_rmse, sarima_r],
    'ARIMA Monthly': [arima_mape_m, arima_rmse_m, arima_r_m],
    'SARIMA Monthly': [sarima_mape_m, sarima_rmse_m, sarima_r_m],
    'autoARIMA Monthly': [autoarima_mape_m, autoarima_rmse_m, autoarima_r_m]
}

compare_df = pd.DataFrame(compare_df, index=['MAPE', 'RMSE', 'R2'])
compare_df = compare_df.transpose()  # Transpose for better readability
compare_df

## Deep Learning Models

### Data Preprocessing

In [ ]:
# Importing necessary libraries
import gc  # Garbage collection for memory management

In [ ]:
# Display the first few rows of the dataframe
# data = data[['Global_active_power', 'Global_reactive_power', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']]
data.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
Date_time,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.000
2006-12-16 17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.000
2006-12-16 17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.000
2006-12-16 17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.000
2006-12-16 17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.000


In [ ]:
## Scaling data
from sklearn.preprocessing import MinMaxScaler

# Define the scalers
scaler = MinMaxScaler()
target_transformer = MinMaxScaler() # For the target variable


data[['Global_active_power', 'Global_reactive_power', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']] = scaler.fit_transform(
    data[['Global_active_power', 'Global_reactive_power', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']]
)

In [ ]:
data.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
Date_time,,,,,,,
2006-12-16 17:24:00,0.375,0.301,234.840,0.378,0.000,0.013,0.548
2006-12-16 17:25:00,0.478,0.314,233.630,0.473,0.000,0.013,0.516
2006-12-16 17:26:00,0.480,0.358,233.290,0.473,0.000,0.025,0.548
2006-12-16 17:27:00,0.481,0.361,233.740,0.473,0.000,0.013,0.548
2006-12-16 17:28:00,0.325,0.380,235.680,0.324,0.000,0.013,0.548


In [ ]:
# Split the data into training and testing sets
train_size = int(len(data) * 0.8)  # 80% training data, 20% testing data
d1_train, d1_test = data.iloc[:train_size], data.iloc[train_size:]

# Display training set information
print(f'Train shape: {d1_train.shape}')

# Display testing set information
print(f'Test shape: {d1_test.shape}')

Train shape: (1660207, 7)
Test shape: (415052, 7)


In [ ]:
d1_train.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
Date_time,,,,,,,
2006-12-16 17:24:00,0.375,0.301,234.840,0.378,0.000,0.013,0.548
2006-12-16 17:25:00,0.478,0.314,233.630,0.473,0.000,0.013,0.516
2006-12-16 17:26:00,0.480,0.358,233.290,0.473,0.000,0.025,0.548
2006-12-16 17:27:00,0.481,0.361,233.740,0.473,0.000,0.013,0.548
2006-12-16 17:28:00,0.325,0.380,235.680,0.324,0.000,0.013,0.548


In [ ]:
d1_test.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
Date_time,,,,,,,
2010-02-11 15:31:00,0.024,0.073,250.090,0.025,0.000,0.025,0.032
2010-02-11 15:32:00,0.023,0.073,249.870,0.025,0.000,0.013,0.032
2010-02-11 15:33:00,0.023,0.072,249.790,0.025,0.000,0.025,0.032
2010-02-11 15:34:00,0.024,0.072,249.870,0.025,0.000,0.013,0.000
2010-02-11 15:35:00,0.023,0.071,248.280,0.025,0.000,0.013,0.032


In [ ]:
## Visualize the distribution of features

# Define the columns for which to plot boxplots
f_columns = ['Global_active_power', 'Global_reactive_power', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2',
       'Sub_metering_3']

# Plot boxplots for each feature
plt.figure(figsize=(15, 15))
for i, column in enumerate(f_columns, 1):
    plt.subplot(2, 3, i)
    sns.boxplot(y=data[column])
    plt.title(f'Boxplot of {column.capitalize()}')

plt.tight_layout()
plt.show()

KeyboardInterrupt: 

### Simple RNN Model

In [ ]:
## Creating a dataset suitable for RNN input
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [ ]:
d1_train.columns

Index(['Global_active_power', 'Global_reactive_power', 'Global_intensity',
       'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3'],
      dtype='object')

In [ ]:
## Manual Sequence Creation
time_steps = 10 # Use the past 10 time steps to predict the value at the next time step

X, y = d1_train[['Global_reactive_power',
       'Global_intensity', 'Sub_metering_1', 'Sub_metering_2',
       'Sub_metering_3']], d1_train['Global_active_power']

X_train, y_train = create_dataset(X, y, time_steps)
X_test, y_test = create_dataset(d1_test[['Global_reactive_power',
       'Global_intensity', 'Sub_metering_1', 'Sub_metering_2',
       'Sub_metering_3']], d1_test['Global_active_power'], time_steps)

# Check the shape of the datasets
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_train shape: (1660197, 10, 5), y_train shape: (1660197,)
X_test shape: (415042, 10, 5), y_test shape: (415042,)


In [ ]:
# ## Automated Sequence Creation
# from keras.preprocessing.sequence import TimeseriesGenerator

# # Define parameters
# time_steps = 10  # Use the past 10 time steps to predict the value at the next time step
# batch_size = 32  # Example batch size

# # Prepare the training data
# train_data = d1_train[['Global_reactive_power', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3', 'Global_active_power']].values

# # Separate features and target for training
# X_train, y_train = train_data[:, :-1], train_data[:, -1]

# # Create TimeseriesGenerator for training
# train_generator = TimeseriesGenerator(X_train, y_train, length=time_steps, batch_size=batch_size)

# # Prepare the test data
# test_data = d1_test[['Global_reactive_power', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3', 'Global_active_power']].values

# # Separate features and target for testing
# X_test, y_test = test_data[:, :-1], test_data[:, -1]

# # Create TimeseriesGenerator for testing
# test_generator = TimeseriesGenerator(X_test, y_test, length=time_steps, batch_size=batch_size)

# # Check the shape of the first batch in the generators
# X_train_batch, y_train_batch = train_generator[0]
# X_test_batch, y_test_batch = test_generator[0]

# print(f"X_train batch shape: {X_train_batch.shape}, y_train batch shape: {y_train_batch.shape}")
# print(f"X_test batch shape: {X_test_batch.shape}, y_test batch shape: {y_test_batch.shape}")


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import RobustScaler, MinMaxScaler
import matplotlib.pyplot as plt

# Build the RNN model
rnn_model = Sequential()
rnn_model.add(SimpleRNN(100, activation='tanh', input_shape=(time_steps, X_train.shape[2])))
rnn_model.add(Dense(1))
rnn_model.compile(optimizer='adam', loss='mse')

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with early stopping
history = rnn_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32, callbacks=[early_stopping])

# Evaluate the model
loss = rnn_model.evaluate(X_test, y_test)
print(f'Validation Loss: {loss}')

Epoch 1/30
 1000/51882 [..............................] - ETA: 8:21 - loss: 0.0013

KeyboardInterrupt: 

In [ ]:
# Display the model summary
rnn_model.summary()

In [ ]:
# Plot training & validation loss values
plt.figure(figsize=(15, 8))
plt.plot(history.history['loss'], label='Train loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.show()

In [ ]:
# Make predictions
rnn_pred = rnn_model.predict(X_test)

# Inverse transform the predictions to the original scale
rnn_pred = target_transformer.inverse_transform(rnn_pred)

# Inverse transform the true values for comparison
y_test = y_test.reshape(-1, 1)  # Ensure y_test is in the correct shape
y_test = target_transformer.inverse_transform(y_test)

# Display the first few predictions and actual values for comparison
pred_df = pd.DataFrame({'Actual': y_test.flatten(), 'Predicted': rnn_pred.flatten()})
print(pred_df.head())

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

# Calculate evaluation metrics
rnn_mse = mean_squared_error(y_test, rnn_pred)
rnn_rmse = np.sqrt(rnn_mse)
rnn_mae = mean_absolute_error(y_test, rnn_pred)
rnn_r2 = r2_score(y_test, rnn_pred)

# Print evaluation metrics
print(f'RNN Model Performance:')
# print(f'MSE: {rnn_mse:.3f}')
print(f'RMSE: {rnn_rmse:.3f}')
# print(f'MAE: {rnn_mae:.3f}')
print(f'R²: {rnn_r2:.3f}')

In [ ]:
# Plotting the results
plt.figure(figsize=(14, 7))
plt.plot(df.index[-len(y_test):], y_test, label='True Values', color='blue')
plt.plot(df.index[-len(y_test):], rnn_pred, label='Predictions', linestyle='dashed', color='red')
plt.xlabel('Date')
plt.ylabel('Mean Temperature')
plt.title('Mean Temperature Predictions vs True Values')
plt.legend()
plt.show()

In [ ]:
# Get training and validation losses from history
training_loss = history.history['loss']
validation_loss = history.history['val_loss']

# Plot loss values over epochs
plt.figure(figsize=(14, 7))
plt.plot(training_loss, label='Training Loss')
plt.plot(validation_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)  # Add grid for better readability
plt.show()


### LSTM Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

# Define parameters
time_steps = 10  # Use the past 10 time steps to predict the value at the next time step
batch_size = 32  # Example batch size
epochs = 5  # Number of epochs for training

In [ ]:
# Prepare the training data
train_data = d1_train[['Global_reactive_power', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3', 'Global_active_power']].values

# Separate features and target for training
X_train, y_train = train_data[:, :-1], train_data[:, -1]

# Create TimeseriesGenerator for training
train_generator = TimeseriesGenerator(X_train, y_train, length=time_steps, batch_size=batch_size)


In [ ]:
# Prepare the test data
test_data = d1_test[['Global_reactive_power', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3', 'Global_active_power']].values

# Separate features and target for testing
X_test, y_test = test_data[:, :-1], test_data[:, -1]

# Create TimeseriesGenerator for testing
test_generator = TimeseriesGenerator(X_test, y_test, length=time_steps, batch_size=batch_size)


In [ ]:
X_train.shape
# Number of Sequences (1660197)
# Number of Time Steps (10)
# Number of Features (5)

(1660207, 5)

- Shape after Using TimeseriesGenerator
When you use TimeseriesGenerator:

`(num_sequences, time_steps, num_features).`

- LSTM input_shape = `(10, 5)`

In [ ]:
# Define the LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(100, activation='tanh', return_sequences=True, input_shape=(time_steps, X_train.shape[1])))
lstm_model.add(LSTM(50, activation='tanh'))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mse')

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with early stopping
history = lstm_model.fit(train_generator, epochs=epochs, validation_data=test_generator, batch_size=batch_size, callbacks=[early_stopping])

# Evaluate the model
loss = lstm_model.evaluate(test_generator)
print(f'Validation Loss: {loss}')

Epoch 1/5
51882/51882 [==============================] - 377s 7ms/step - loss: 6.2471e-04 - val_loss: 3.5812e-04
Epoch 2/5
51882/51882 [==============================] - 375s 7ms/step - loss: 5.3494e-04 - val_loss: 3.6001e-04
Epoch 3/5
51882/51882 [==============================] - 368s 7ms/step - loss: 5.0654e-04 - val_loss: 3.6580e-04
Epoch 4/5
51882/51882 [==============================] - 371s 7ms/step - loss: 4.8649e-04 - val_loss: 3.3450e-04
Epoch 5/5
12971/12971 [==============================] - 44s 3ms/step - loss: 3.2428e-04
Validation Loss: 0.0003242813691031188


In [ ]:
lstm_model.summary()

In [ ]:
# Make predictions
lstm_pred = lstm_model.predict(test_generator)
lstm_pred = target_transformer.inverse_transform(lstm_pred)  # Inverse transform to original scale

In [ ]:
# Inverse transform the true values for comparison
y_test = y_test.reshape(-1, 1)
y_test = target_transformer.inverse_transform(y_test)

In [ ]:
# Calculate RMSE and R2 scores
rmse = np.sqrt(mean_squared_error(y_test, lstm_pred))
r2 = r2_score(y_test, lstm_pred)

print(f'RMSE: {rmse}')
print(f'R2 Score: {r2}')

In [ ]:
# Plot the results
plt.figure(figsize=(14, 7))
plt.plot(df.index[-len(y_test):], y_test, label='True Values')
plt.plot(df.index[-len(y_test):], lstm_pred, label='Predictions', linestyle='dashed')
plt.xlabel('Date')
plt.ylabel('Global_reactive_power')
plt.title('Global_reactive_power Predictions vs True Values')
plt.legend()
plt.show()

In [ ]:
# Plotting training and validation loss
training_loss = history.history['loss']
validation_loss = history.history['val_loss']

plt.figure(figsize=(10, 6))
plt.plot(training_loss, label='Training Loss')
plt.plot(validation_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

### Bidirectional LSTM

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

# Define parameters
time_steps = 10  # Use the past 10 time steps to predict the value at the next time step
batch_size = 32  # Example batch size
epochs = 5  # Number of epochs for training

In [ ]:
# Prepare the training data
train_data = d1_train[['Global_reactive_power', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3', 'Global_active_power']].values

# Separate features and target for training
X_train, y_train = train_data[:, :-1], train_data[:, -1]

# Create TimeseriesGenerator for training
train_generator = TimeseriesGenerator(X_train, y_train, length=time_steps, batch_size=batch_size)


In [ ]:
# Prepare the test data
test_data = d1_test[['Global_reactive_power', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3', 'Global_active_power']].values

# Separate features and target for testing
X_test, y_test = test_data[:, :-1], test_data[:, -1]

# Create TimeseriesGenerator for testing
test_generator = TimeseriesGenerator(X_test, y_test, length=time_steps, batch_size=batch_size)


In [ ]:
X_train.shape
# Number of Sequences (1660197)
# Number of Time Steps (10)
# Number of Features (5)

In [ ]:
# Define the LSTM model
blstm_model = Sequential()
blstm_model.add(Bidirectional(LSTM(100, activation='tanh', return_sequences=True, input_shape=(time_steps, X_train.shape[1]))))
blstm_model.add(Dense(1))
blstm_model.compile(optimizer='adam', loss='mse')

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with early stopping
history = blstm_model.fit(train_generator, epochs=epochs, validation_data=test_generator, batch_size=batch_size, callbacks=[early_stopping])

# Evaluate the model
loss = blstm_model.evaluate(test_generator)
print(f'Validation Loss: {loss}')

In [ ]:
blstm_model.summary()

In [ ]:
# Make predictions
blstm_pred = lstm_model.predict(test_generator)
blstm_pred = target_transformer.inverse_transform(blstm_pred)  # Inverse transform to original scale

In [ ]:
# Inverse transform the true values for comparison
y_test = y_test.reshape(-1, 1)
y_test = target_transformer.inverse_transform(y_test)

In [ ]:
# Calculate RMSE and R2 scores
rmse = np.sqrt(mean_squared_error(y_test, blstm_pred))
r2 = r2_score(y_test, blstm_pred)

print(f'RMSE: {rmse}')
print(f'R2 Score: {r2}')

In [ ]:
# Plot the results
plt.figure(figsize=(14, 7))
plt.plot(df.index[-len(y_test):], y_test, label='True Values')
plt.plot(df.index[-len(y_test):], lstm_pred, label='Predictions', linestyle='dashed')
plt.xlabel('Date')
plt.ylabel('Mean Temperature')
plt.title('Mean Temperature Predictions vs True Values')
plt.legend()
plt.show()

In [ ]:
# Plotting training and validation loss
training_loss = history.history['loss']
validation_loss = history.history['val_loss']

plt.figure(figsize=(10, 6))
plt.plot(training_loss, label='Training Loss')
plt.plot(validation_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

### GRU Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, GRU, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

# Define parameters
time_steps = 10  # Use the past 10 time steps to predict the value at the next time step
batch_size = 32  # Example batch size
epochs = 1  # Number of epochs for training

In [ ]:
# Fit a MinMaxScaler on the training data
scaler = MinMaxScaler()

scaler.fit_transform(d1_train)
scaler.transform(d1_test)

array([[0.02353793, 0.07338129, 0.86882068, ..., 0.        , 0.025     ,
        0.03225806],
       [0.02335687, 0.07338129, 0.86171244, ..., 0.        , 0.0125    ,
        0.03225806],
       [0.02335687, 0.07194245, 0.85912763, ..., 0.        , 0.025     ,
        0.03225806],
       ...,
       [0.0780373 , 0.        , 0.53699515, ..., 0.        , 0.        ,
        0.        ],
       [0.07767518, 0.        , 0.53311793, ..., 0.        , 0.        ,
        0.        ],
       [0.07749412, 0.        , 0.52827141, ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
# Prepare the training data
train_data = d1_train[['Global_reactive_power', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3', 'Global_active_power']].values

# Separate features and target for training
X_train, y_train = train_data[:, :-1], train_data[:, -1]

# Create TimeseriesGenerator for training
train_generator = TimeseriesGenerator(X_train, y_train, length=time_steps, batch_size=batch_size)


In [ ]:
# Prepare the test data
test_data = d1_test[['Global_reactive_power', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3', 'Global_active_power']].values

# Separate features and target for testing
X_test, y_test = test_data[:, :-1], test_data[:, -1]

# Create TimeseriesGenerator for testing
test_generator = TimeseriesGenerator(X_test, y_test, length=time_steps, batch_size=batch_size)


In [ ]:
X_train.shape
# Number of Sequences (1660197)
# Number of Time Steps (10)
# Number of Features (5)

(1660207, 5)

In [ ]:
# Define the LSTM model
gru_model = Sequential()
gru_model.add(Bidirectional(GRU(100, activation='tanh', return_sequences=True, input_shape=(time_steps, X_train.shape[1]))))
gru_model.add(Dense(1))
gru_model.compile(optimizer='adam', loss='mse')

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with early stopping
history = gru_model.fit(train_generator, epochs=epochs, validation_data=test_generator, batch_size=batch_size, callbacks=[early_stopping])

# Evaluate the model
loss = gru_model.evaluate(test_generator)
print(f'Validation Loss: {loss}')

43621/51882 [========================>.....] - ETA: 2:17 - loss: 0.0030

In [ ]:
gru_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_2 (Bidirecti  (None, None, 200)         64200     
 onal)                                                           
                                                                 
 dense_2 (Dense)             (None, None, 1)           201       
                                                                 
Total params: 64401 (251.57 KB)
Trainable params: 64401 (251.57 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Make predictions
gru_model_pred = gru_model.predict(test_generator)
gru_model_pred = target_transformer.inverse_transform(gru_model_pred)  # Inverse transform to original scale

12971/12971 [==============================] - 83s 6ms/step


NotFittedError: This MinMaxScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
# Inverse transform the true values for comparison
y_test = y_test.reshape(-1, 1)
y_test = target_transformer.inverse_transform(y_test)

In [ ]:
# Calculate RMSE and R2 scores
rmse = np.sqrt(mean_squared_error(y_test, model_pred))
r2 = r2_score(y_test, model_pred)

print(f'RMSE: {rmse}')
print(f'R2 Score: {r2}')

In [ ]:
# Plot the results
plt.figure(figsize=(14, 7))
plt.plot(df.index[-len(y_test):], y_test, label='True Values')
plt.plot(df.index[-len(y_test):], lstm_pred, label='Predictions', linestyle='dashed')
plt.xlabel('Date')
plt.ylabel('Global_reactive_power')
plt.title('Global_reactive_power Predictions vs True Values')
plt.legend()
plt.show()

In [ ]:
# Plotting training and validation loss
training_loss = history.history['loss']
validation_loss = history.history['val_loss']

plt.figure(figsize=(10, 6))
plt.plot(training_loss, label='Training Loss')
plt.plot(validation_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

### Compare Models (For daily forecast)

In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score

In [ ]:
print(pred_df.head())

In [ ]:
print(compare_df.head())

In [ ]:
# Calculate metrics for each model
def calculate_metrics(true_values, predicted_values):
    mape = mean_absolute_percentage_error(true_values, predicted_values)
    r2 = r2_score(true_values, predicted_values)
    rmse = math.sqrt(mean_squared_error(true_values, predicted_values))
    return mape, rmse, r2

# Metrics for each model
rnn_mape, rnn_rmse, rnn_r = calculate_metrics(y_test, rnn_pred)
lstm_mape, lstm_rmse, lstm_r = calculate_metrics(y_test, lstm_pred)
bilstm_mape, bilstm_rmse, bilstm_r = calculate_metrics(y_test, blstm_pred)
gru_mape, gru_rmse, gru_r = calculate_metrics(y_test, gru_pred)

In [ ]:
# Dataframe for deep learning models comparison
dl_compare = {
    'RNN daily': [rnn_mape, rnn_rmse, rnn_r],
    'LSTM daily': [lstm_mape, lstm_rmse, lstm_r],
    'BiLSTM daily': [bilstm_mape, bilstm_rmse, bilstm_r],
    'GRU daily': [gru_mape, gru_rmse, gru_r]
}
dl_compare_df = pd.DataFrame(dl_compare, index=['MAE', 'RMSE', 'R2'])

# Concatenate with existing compare_df
compare_models_df = pd.concat([compare_df, dl_compare_df], axis=1)
compare_models_df

In [ ]:
# Data for comparison plot
data = {
    'Model': ['Prophet daily', 'ARIMA daily', 'SARIMA daily', 'RNN daily', 'LSTM daily', 'BiLSTM daily', 'GRU daily'],
    'MAE': [0.052, 0.245, 0.143, rnn_mape, lstm_mape, bilstm_mape, gru_mape],
    'RMSE': [1.718, 8.100, 4.803, rnn_rmse, lstm_rmse, bilstm_rmse, gru_rmse],
    'R2': [0.908, -1.050, 0.279, rnn_r, lstm_r, bilstm_r, gru_r]
}
compare = pd.DataFrame(data)

# Function to add labels to the bar plots
def add_labels(ax):
    for p in ax.patches:
        width = p.get_width()
        ax.text(width, p.get_y() + p.get_height() / 2, '{:.3f}'.format(width), ha='left', va='center')

# Plotting
plt.figure(figsize=(15, 14))

plt.subplot(411)
compare_sorted = compare.sort_values(by="R2", ascending=False)
ax = sns.barplot(x="R2", y="Model", data=compare_sorted, palette="Blues_d")
add_labels(ax)
plt.title('R2 Scores')

plt.subplot(412)
compare_sorted = compare.sort_values(by="MAE", ascending=False)
ax = sns.barplot(x="MAE", y="Model", data=compare_sorted, palette="Blues_d")
add_labels(ax)
plt.title('MAE Scores')

plt.subplot(413)
compare_sorted = compare.sort_values(by="RMSE", ascending=False)
ax = sns.barplot(x="RMSE", y="Model", data=compare_sorted, palette="Blues_d")
add_labels(ax)
plt.title('RMSE Scores')

plt.tight_layout()
plt.show()